Imports


In [20]:
import csv
import io
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\llose\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\llose\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Create datasets

## Importing Data

In [2]:

import csv
with open('dataset.csv') as csvfile:
  spamreader = csv.reader(csvfile,delimiter=';')
  donnees=[]
  for row in spamreader:
    review=[""]
    for part in row:
        if part!='':
            review[0] += part
    donnees.append(review[0])

In [3]:
#charge les étiquettes du jeu de données

with open('labels.csv') as csvfile:
  spamreader = csv.reader(csvfile,delimiter=';')
  labels=[]
  for row in spamreader:
   labels.append(int(row[0]))

In [4]:
print(labels)
print(len(labels))
print(len(donnees))
#Il manque deux avis... De plus on note que tous les 5000 premiers avis sont négatifs et les 5000 suivants sont positifs

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,

In [5]:
print(labels.count(-1))
#le résultat est 5000. On a 5000 avis positifs et 5000 avis négatifs
print(donnees[4997])
print(donnees[4998])
#L'avis 4997 est négatif alors que l'avis 4998 est positif. Ainsi, se sont donc 2 avis négatifs qui ont été supprimés.

5000
I've seen other Guinea Pig films, like Mermaid in a Manhole & He Never Dies, and while they're pretty sick, they at least have a bit of a sense of humor to them (however dark). Devil's Experiment though, is nothing more than filming a bunch of punks submitting a young girl to many methods of torture and violence and there's no plot and no redeeming values to it whatsoever. It isn't remotely scary, except for perhaps the mind-set, but it's definitely disgusting and I certainly would not call it entertainment. Now, I'm not the paragon of good taste in films and I like my trash and sleaze, but this was too much for me, really. I'm all for disturbing, in fact, I seek out things that are disturbing whenever possible, but this was a bit much. The young woman is kicked, beaten, submitted to continuous noise via headphones until she drools, and then pelted with raw meat and innards and left hanging in a hammock out in the woods somewhere. If you think that sounds great, go for it, but it 

In [6]:
labels=labels[2:]
print(len(labels))

9998


## Shuffling the observations


In [7]:
#On veut que les avis positifs et négatifs soient répartis uniformément dans l'échantillon pour la cross-validation
import random
random.seed(100)

indexes=list(range(9998))
shuffled_indexes=random.sample(indexes,len(indexes))
print(shuffled_indexes)

[2386, 7528, 7453, 2863, 6439, 5730, 7102, 8304, 1801, 8731, 1987, 1313, 7470, 4315, 784, 3350, 5494, 3763, 5069, 3340, 2923, 2307, 3089, 5685, 6063, 6712, 3454, 6603, 7554, 9099, 4518, 6145, 2628, 2033, 2959, 92, 9875, 6480, 2418, 9275, 2663, 3156, 2736, 428, 3886, 7343, 6351, 2074, 9075, 859, 4048, 8308, 4748, 9672, 5549, 8653, 5624, 8887, 5947, 7922, 6582, 1014, 1660, 4913, 7118, 4137, 3928, 7166, 8414, 8085, 9546, 4164, 2411, 953, 9590, 5353, 2698, 894, 4508, 1982, 831, 9754, 3590, 5076, 8593, 3546, 9309, 5852, 5468, 7466, 205, 1868, 7790, 3159, 8606, 1951, 9759, 3426, 719, 6387, 1704, 6563, 2482, 7810, 8930, 636, 7294, 2639, 808, 7763, 771, 3246, 4076, 2579, 1545, 639, 2293, 3468, 1477, 2015, 4122, 1094, 6586, 603, 8856, 6984, 7768, 7381, 5630, 7441, 8578, 3181, 9498, 4115, 6844, 2381, 3419, 6762, 1268, 9795, 4407, 6566, 4267, 1555, 3120, 3729, 775, 6398, 4446, 46, 4444, 4988, 618, 427, 9149, 4825, 3800, 6597, 159, 1310, 3599, 4882, 377, 1422, 9587, 5777, 5309, 7279, 552, 3019, 13

In [8]:
shuffled_donnees=[]
shuffled_labels=[]
for i in shuffled_indexes:
  shuffled_donnees.append(donnees[i])
  shuffled_labels.append(labels[i])

In [9]:
labels=shuffled_labels
donnees=shuffled_donnees

In [10]:
vanilla_donnees=[]
for d in donnees:
  vanilla_donnees.append(nltk.word_tokenize(d))
print(vanilla_donnees[3])

['Rachel', ',', 'Jo', ',', 'Hannah', ',', 'Tina', ',', 'Bradley', 'and', 'John', 'are', 'all', 'on', 'top', 'form', 'here', '.', 'They', 'deserve', 'oscar', 'nominations', 'for', 'their', 'performances', '.', 'I', 'am', 'a', 'great', 'fan', 'of', 'the', 'tv', 'show', 'aswell', '.', 'Their', 'music', 'rocks', 'and', 'they', "'re", 'all', 'so', 'talented', '!', 'I', 'am', 'also', 'a', 'great', 'exponent', 'of', 'SARCASM', '!', '!', '!', '!', '!', '!', 'IF', 'YOU', "'RE", 'AN', 'S', 'CLUB', 'FAN', 'DO', 'NOT', 'READ', 'THIS', '!', '!', '!', '!', '!', 'The', 'performances', 'are', 'terribly', 'weak', ',', 'the', 'dialogue', 'is', 'terrible', 'and', 'the', 'jokes', 'are', 'not', 'even', 'executed', 'properly', '(', 'i', 'feel', 'sorry', 'for', 'the', 'director', ')', '.', 'The', 'jokes', 'are', 'so', 'unbelievably', 'bad', 'that', '8', 'little', ',', 'passionate', 'S', 'Club', 'fans', 'were', "n't", 'laughing', '.', 'They', 'thought', 'they', 'could', 'do', 'it', 'better', '.', 'And', 'they

# Text Pretreatment

## Lemmatization

In [17]:
#take a csv cell and return a big string lemantize 
import nltk
nltk.download('punkt')
nltk.download('wordnet')
def lematization(review):
    # Init the Wordnet Lemmatizer
    lemmatizer = nltk.WordNetLemmatizer()
    #transform all the strings in one big string
    #makeOneBigString = ' '.join(csvCell)
    #split the sentence in single words
    word_list = nltk.word_tokenize(review)
    #lemmatize and join the words in one big string again
    lemmatized_output = ' '.join(lemmatizer.lemmatize(w) for w in word_list)
    return lemmatized_output

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\llose\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\llose\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
lemmatized_reviews=[]
for d in donnees:
  lemmatized_reviews.append(lematization(d))
print(lemmatized_reviews[54])

My Tutor Friend is a well scripted romance comedy movie that ha something similar to My Sassy Girl .. there 's no kissing/sex scene . Hollywood should learn more from Korean production . Sex is not always required in a good romantic movie.The movie is of light hearted tone with occasional cartoon CG scene blended into the movie . I like the part when Ji-Hoon almost kissed Su-Wan . The funniest moment is when Ji-Hoon punched Su-Wan 's first love because he dumped Su-Wan for another girl and he is going to be a father soon . How he became a father wa revealed in the next scene , which brings smile to the audience.Mao point : 8/10


## Tree-tagger



In [11]:
#N'exécuter qu'une seule fois!
!pip install treetaggerwrapper

You should consider upgrading via the 'c:\users\llose\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [45]:
nltk.download('averaged_perceptron_tagger')
# test=nltk.word_tokenize(donnees[0])
# nltk.pos_tag(test)
tagged_reviews=[]
def tagger(review):
  lemmatized_word_list=[]
  lemmatizer = nltk.WordNetLemmatizer()
  word_list = nltk.word_tokenize(review)
  for w in word_list:
      lemmatized_word_list.append(lemmatizer.lemmatize(w))
  return nltk.pos_tag(lemmatized_word_list)
  

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\llose\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [37]:
def tagger(review):
  lemmatized_word_list=[]

  lemmatizer = nltk.WordNetLemmatizer()
  word_list = nltk.word_tokenize(review)
  for w in word_list:
    lemmatized_word_list.append(lemmatizer.lemmatize(w))
  return ' '.join(w[0]+' '+w[1] for w in nltk.pos_tag(lemmatized_word_list))

In [46]:
tagged_reviews=[]
for d in donnees:
  tagged_reviews.append(tagger(d.lower()))


In [47]:
print(tagged_reviews[54])

[('my', 'PRP$'), ('tutor', 'NN'), ('friend', 'NN'), ('is', 'VBZ'), ('a', 'DT'), ('well', 'RB'), ('scripted', 'VBN'), ('romance', 'NN'), ('comedy', 'NN'), ('movie', 'NN'), ('that', 'IN'), ('ha', 'VBZ'), ('something', 'NN'), ('similar', 'JJ'), ('to', 'TO'), ('my', 'PRP$'), ('sassy', 'JJ'), ('girl', 'NN'), ('..', 'NNP'), ('there', 'EX'), ("'s", 'VBZ'), ('no', 'DT'), ('kissing/sex', 'JJ'), ('scene', 'NN'), ('.', '.'), ('hollywood', 'NN'), ('should', 'MD'), ('learn', 'VB'), ('more', 'JJR'), ('from', 'IN'), ('korean', 'JJ'), ('production', 'NN'), ('.', '.'), ('sex', 'NN'), ('is', 'VBZ'), ('not', 'RB'), ('always', 'RB'), ('required', 'VBN'), ('in', 'IN'), ('a', 'DT'), ('good', 'JJ'), ('romantic', 'JJ'), ('movie.the', 'NN'), ('movie', 'NN'), ('is', 'VBZ'), ('of', 'IN'), ('light', 'JJ'), ('hearted', 'VBN'), ('tone', 'NN'), ('with', 'IN'), ('occasional', 'JJ'), ('cartoon', 'NN'), ('cg', 'NNS'), ('scene', 'NN'), ('blended', 'VBD'), ('into', 'IN'), ('the', 'DT'), ('movie', 'NN'), ('.', '.'), ('i',

## Bi-Grams

In [11]:
from nltk.util import ngrams
import re
import string

def get_ngrams(text, n):
    ngramnums = nltk.word_tokenize(text)
    ll = [x for x in ngramnums if not re.fullmatch('[' + string.punctuation + ']+', x)]
    ll = ngrams(ll, n)
    return [' '.join(grams) for grams in ll]


 # On défini ici notre phrase sur laquelle on applique le ngrams en 1er paramètre
# Dans le second paramètre, rentrer la valeure de n.
ngram = get_ngrams("U. S. Rep. David Cicilline says that when he was mayor of Providence, the city’s internal auditor was not blocked from accessing financial records", 3)
for gram in ngram:
    print(ngram)

['U. S. Rep.', 'S. Rep. David', 'Rep. David Cicilline', 'David Cicilline says', 'Cicilline says that', 'says that when', 'that when he', 'when he was', 'he was mayor', 'was mayor of', 'mayor of Providence', 'of Providence the', 'Providence the city', 'the city ’', 'city ’ s', '’ s internal', 's internal auditor', 'internal auditor was', 'auditor was not', 'was not blocked', 'not blocked from', 'blocked from accessing', 'from accessing financial', 'accessing financial records']
['U. S. Rep.', 'S. Rep. David', 'Rep. David Cicilline', 'David Cicilline says', 'Cicilline says that', 'says that when', 'that when he', 'when he was', 'he was mayor', 'was mayor of', 'mayor of Providence', 'of Providence the', 'Providence the city', 'the city ’', 'city ’ s', '’ s internal', 's internal auditor', 'internal auditor was', 'auditor was not', 'was not blocked', 'not blocked from', 'blocked from accessing', 'from accessing financial', 'accessing financial records']
['U. S. Rep.', 'S. Rep. David', 'Rep

In [12]:
bigrams_reviews=[]
for d in donnees:
    bigrams_reviews.append(get_ngrams(d.lower(),2))


# Classification Model

## TF-IDF vectorizing

In [13]:
def dummy_fun(doc):
    return doc

In [15]:
#Les mots qui apparaissent dans plus de 80% des documents sont ignorés.
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
from nltk.corpus import stopwords
vectorizer = TfidfVectorizer (max_features=5000,max_df=0.8, stop_words=stopwords.words('english'))
vec2 = TfidfVectorizer (max_features=5000,max_df=0.8, stop_words=stopwords.words('english'),analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\llose\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
vanilla_tfidf = vectorizer.fit_transform(donnees).toarray()

In [54]:
lemmatized_tfidf=vectorizer.fit_transform(lemmatized_reviews).toarray()

In [70]:
tagger_tfidf=tfidf.fit_transform(tagged_reviews).toarray()

In [18]:
vec3 = TfidfVectorizer (max_features=5000,max_df=0.8, stop_words=stopwords.words('english'),analyzer='word',
    tokenizer=dummy_fun,preprocessor=dummy_fun,token_pattern=None,ngram_range=(1,2))
bigrams_tfidf=vec3.fit_transform(bigrams_reviews).toarray()

In [13]:
len(vanilla_tfidf[0])

5000

## Vanilla sample


### K-Nearest Neighbors

In [120]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(vanilla_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(vanilla_tfidf[i])
        y_train.append(labels[i])

In [121]:

knn=KNeighborsClassifier()
parameters={'n_neighbors':list(range(10,21)),'p':[2],'n_jobs':[-1]}

In [122]:
clf = GridSearchCV(knn,parameters,cv=3,verbose=10)
clf.fit(x_train,y_train)

Fitting 3 folds for each of 11 candidates, totalling 33 fits
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... n_jobs=-1, n_neighbors=10, p=2, score=0.781, total=  38.4s
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   38.3s remaining:    0.0s


[CV] ...... n_jobs=-1, n_neighbors=10, p=2, score=0.780, total=  36.3s
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................


...

[CV] ...... n_jobs=-1, n_neighbors=12, p=2, score=0.779, total=  35.2s
[CV] n_jobs=-1, n_neighbors=13, p=2 ..................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  6.0min remaining:    0.0s


[CV] ...... n_jobs=-1, n_neighbors=13, p=2, score=0.804, total=  38.4s
[CV] n_jobs=-1, n_neighbors=13, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=13, p=2, score=0.808, total=  31.7s
[CV] n_jobs=-1, n_neighbors=13, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=13, p=2, score=0.789, total=  31.6s
[CV] n_jobs=-1, n_neighbors=14, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=14, p=2, score=0.794, total=  31.6s
[CV] n_jobs=-1, n_neighbors=14, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=14, p=2, score=0.803, total=  31.9s
[CV] n_jobs=-1, n_neighbors=14, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=14, p=2, score=0.781, total=  32.0s
[CV] n_jobs=-1, n_neighbors=15, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=15, p=2, score=0.809, total=  32.1s
[CV] n_jobs=-1, n_neighbors=15, p=2 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed: 18.9min finished


GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_jobs': [-1],
                         'n_neighbors': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                         20],
                         'p': [2]},
             verbose=10)

In [123]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1246  277]
 [ 265 1212]]
              precision    recall  f1-score   support

          -1       0.82      0.82      0.82      1523
           1       0.81      0.82      0.82      1477

    accuracy                           0.82      3000
   macro avg       0.82      0.82      0.82      3000
weighted avg       0.82      0.82      0.82      3000

0.8193333333333334
{'n_jobs': -1, 'n_neighbors': 19, 'p': 2}


### Naïve Bayes

In [69]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(vanilla_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(vanilla_tfidf[i])
        y_train.append(labels[i])

In [73]:
print(x_train[0])

[0. 0. 0. ... 0. 0. 0.]


In [70]:

nb=ComplementNB()
parameters={'alpha':np.arange(0.1,1,0.01),'norm':[True,False]}

In [71]:
clf = GridSearchCV(nb,parameters,verbose=100)
clf.fit(x_train,y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.848, total=   0.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.850, total=   0.2s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.859, total=   0.2s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.836, total=   0.2s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elap

[CV] .............. alpha=0.13, norm=False, score=0.896, total=   0.2s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    7.3s remaining:    0.0s
[CV] alpha=0.13, norm=False ..........................................
[CV] .............. alpha=0.13, norm=False, score=0.894, total=   0.2s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:    7.5s remaining:    0.0s
[CV] alpha=0.13, norm=False ..........................................
[CV] .............. alpha=0.13, norm=False, score=0.876, total=   0.2s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:    7.7s remaining:    0.0s
[CV] alpha=0.13, norm=False ..........................................
[CV] .............. alpha=0.13, norm=False, score=0.889, total=   0.2s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    7.9s remaining:    0.0s
[CV] alpha=0.13999999999999999, norm=True ............................
[CV]  alpha=0.13999999999999999, norm=True, score=0.852, total=   0.2s
[Parallel(n_jobs=1)]: Done  41 out of  41

[CV]  alpha=0.16999999999999998, norm=True, score=0.843, total=   0.2s
[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:   14.7s remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=True ............................
[CV]  alpha=0.16999999999999998, norm=True, score=0.853, total=   0.2s
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:   14.9s remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=False ...........................
[CV]  alpha=0.16999999999999998, norm=False, score=0.885, total=   0.2s
[Parallel(n_jobs=1)]: Done  76 out of  76 | elapsed:   15.1s remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=False ...........................
[CV]  alpha=0.16999999999999998, norm=False, score=0.895, total=   0.2s
[Parallel(n_jobs=1)]: Done  77 out of  77 | elapsed:   15.3s remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=False ...........................
[CV]  alpha=0.16999999999999998, norm=False, score=0.893, total=   0.2s
[Parallel(n_jobs=1)]: Done  78 out of 

[CV]  alpha=0.20999999999999996, norm=False, score=0.895, total=   0.2s
[CV] alpha=0.20999999999999996, norm=False ...........................
[CV]  alpha=0.20999999999999996, norm=False, score=0.892, total=   0.2s
[CV] alpha=0.20999999999999996, norm=False ...........................
[CV]  alpha=0.20999999999999996, norm=False, score=0.877, total=   0.2s
[CV] alpha=0.20999999999999996, norm=False ...........................
[CV]  alpha=0.20999999999999996, norm=False, score=0.890, total=   0.2s
[CV] alpha=0.21999999999999995, norm=True ............................
[CV]  alpha=0.21999999999999995, norm=True, score=0.854, total=   0.2s
[CV] alpha=0.21999999999999995, norm=True ............................
[CV]  alpha=0.21999999999999995, norm=True, score=0.858, total=   0.2s
[CV] alpha=0.21999999999999995, norm=True ............................
[CV]  alpha=0.21999999999999995, norm=True, score=0.865, total=   0.2s
[CV] alpha=0.21999999999999995, norm=True ............................
[C

[CV] . alpha=0.2699999999999999, norm=True, score=0.855, total=   0.2s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.887, total=   0.2s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.898, total=   0.2s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.892, total=   0.2s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.877, total=   0.2s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.889, total=   0.2s
[CV] alpha=0.2799999999999999, norm=True .............................
[CV] . alpha=0.2799999999999999, norm=True, score=0.856, total=   0.2s
[CV] alpha=0.2799999999999999, norm=True .............................
[CV] .

...

[CV]  alpha=0.7899999999999996, norm=False, score=0.895, total=   0.2s
[CV] alpha=0.7899999999999996, norm=False ............................
[CV]  alpha=0.7899999999999996, norm=False, score=0.899, total=   0.2s
[CV] alpha=0.7899999999999996, norm=False ............................
[CV]  alpha=0.7899999999999996, norm=False, score=0.883, total=   0.2s
[CV] alpha=0.7899999999999996, norm=False ............................
[CV]  alpha=0.7899999999999996, norm=False, score=0.893, total=   0.2s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] . alpha=0.7999999999999996, norm=True, score=0.863, total=   0.2s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] . alpha=0.7999999999999996, norm=True, score=0.877, total=   0.2s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] . alpha=0.7999999999999996, norm=True, score=0.875, total=   0.2s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] .

[CV] . alpha=0.8499999999999996, norm=True, score=0.861, total=   0.2s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.886, total=   0.2s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.895, total=   0.2s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.899, total=   0.2s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.883, total=   0.2s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.893, total=   0.2s
[CV] alpha=0.8599999999999995, norm=True .............................
[CV] . alpha=0.8599999999999995, norm=True, score=0.864, total=   0.2s
[CV] alpha=0.8599999999999995, norm=True .............................
[CV] .

[CV] . alpha=0.9099999999999996, norm=True, score=0.877, total=   0.2s
[CV] alpha=0.9099999999999996, norm=True .............................
[CV] . alpha=0.9099999999999996, norm=True, score=0.860, total=   0.2s
[CV] alpha=0.9099999999999996, norm=True .............................
[CV] . alpha=0.9099999999999996, norm=True, score=0.861, total=   0.2s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.886, total=   0.2s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.894, total=   0.2s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.900, total=   0.2s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.884, total=   0.2s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  

[CV] . alpha=0.9699999999999995, norm=True, score=0.864, total=   0.2s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.877, total=   0.2s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.877, total=   0.2s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.862, total=   0.2s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.862, total=   0.2s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  alpha=0.9699999999999995, norm=False, score=0.886, total=   0.2s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  alpha=0.9699999999999995, norm=False, score=0.896, total=   0.2s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  

GridSearchCV(estimator=ComplementNB(),
             param_grid={'alpha': array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ,
       0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31,
       0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42,
       0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53,
       0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64,
       0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75,
       0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86,
       0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97,
       0.98, 0.99]),
                         'norm': [True, False]},
             verbose=100)

In [72]:
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1345  172]
 [ 176 1307]]
              precision    recall  f1-score   support

          -1       0.88      0.89      0.89      1517
           1       0.88      0.88      0.88      1483

    accuracy                           0.88      3000
   macro avg       0.88      0.88      0.88      3000
weighted avg       0.88      0.88      0.88      3000

0.884
{'alpha': 0.9699999999999995, 'norm': False}


### SVM

In [124]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(vanilla_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(vanilla_tfidf[i])
        y_train.append(labels[i])

In [125]:
svm_vectorizer = TfidfVectorizer (max_features=2500,max_df=0.8, stop_words=stopwords.words('english'))
svm_vanilla_tfidf = svm_vectorizer.fit_transform(donnees).toarray()
len(svm_vanilla_tfidf[0])

2500

In [126]:
from sklearn.svm import SVC

svc=SVC()
parameters = {'C': [0.1,1, 10, 100], 'gamma': [1],'kernel': ['rbf', 'sigmoid']}

In [127]:
predictions=clf.predict(x_test)
clf = GridSearchCV(svc,parameters,cv=3,verbose=10)
clf.fit(x_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.798, total= 3.0min
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.795, total= 2.8min
[CV] C=0.1, gamma=1, kernel=rbf ......................................


...

[CV] ............ C=1, gamma=1, kernel=rbf, score=0.903, total= 2.5min
[CV] C=1, gamma=1, kernel=sigmoid ....................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 23.0min remaining:    0.0s


[CV] ........ C=1, gamma=1, kernel=sigmoid, score=0.908, total= 1.3min
[CV] C=1, gamma=1, kernel=sigmoid ....................................
[CV] ........ C=1, gamma=1, kernel=sigmoid, score=0.901, total= 1.3min
[CV] C=1, gamma=1, kernel=sigmoid ....................................
[CV] ........ C=1, gamma=1, kernel=sigmoid, score=0.903, total= 1.3min
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.919, total= 2.8min
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.906, total= 3.7min
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.906, total= 3.8min
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.870, total= 1.5min
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 53.4min finished


GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1],
                         'kernel': ['rbf', 'sigmoid']},
             verbose=10)

In [128]:
#y_test=labels[:3000]
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1339  145]
 [ 107 1409]]
              precision    recall  f1-score   support

          -1       0.93      0.90      0.91      1484
           1       0.91      0.93      0.92      1516

    accuracy                           0.92      3000
   macro avg       0.92      0.92      0.92      3000
weighted avg       0.92      0.92      0.92      3000

0.916
{'C': 10, 'gamma': 1, 'kernel': 'rbf'}


## Lemmatized sample

### KNN

In [116]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(lemmatized_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(lemmatized_tfidf[i])
        y_train.append(labels[i])

In [117]:
knn=KNeighborsClassifier()
parameters={'n_neighbors':list(range(10,21)),'p':[2],'n_jobs':[-1]}

In [118]:
clf = GridSearchCV(knn,parameters,cv=3,verbose=100)
clf.fit(x_train,y_train)

Fitting 3 folds for each of 11 candidates, totalling 33 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=10, p=2, score=0.781, total=  33.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   33.1s remaining:    0.0s
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=10, p=2, score=0.787, total=  43.3s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.3min remaining:    0.0s
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=10, p=2, score=0.774, total=  43.1s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.0min remaining:    0.0s
[CV] n_jobs=-1, n_neighbors=11, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=11, p=2, score=0.793, total=  46.8s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapse

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_jobs': [-1],
                         'n_neighbors': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                         20],
                         'p': [2]},
             verbose=100)

In [119]:
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1263  263]
 [ 299 1175]]
              precision    recall  f1-score   support

          -1       0.81      0.83      0.82      1526
           1       0.82      0.80      0.81      1474

    accuracy                           0.81      3000
   macro avg       0.81      0.81      0.81      3000
weighted avg       0.81      0.81      0.81      3000

0.8126666666666666
{'n_jobs': -1, 'n_neighbors': 19, 'p': 2}


### Naive Bayes

In [104]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(lemmatized_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(lemmatized_tfidf[i])
        y_train.append(labels[i])

In [105]:
nb=ComplementNB()
parameters={'alpha':np.arange(0.1,1,0.01),'norm':[True,False]}

In [106]:
clf = GridSearchCV(nb,parameters,verbose=100)
clf.fit(x_train,y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.851, total=   0.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.849, total=   0.4s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.869, total=   0.4s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.1s remaining:    0.0s
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.836, total=   0.4s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elap

[CV] .............. alpha=0.13, norm=False, score=0.891, total=   0.4s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:   13.5s remaining:    0.0s
[CV] alpha=0.13, norm=False ..........................................
[CV] .............. alpha=0.13, norm=False, score=0.895, total=   0.4s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:   13.9s remaining:    0.0s
[CV] alpha=0.13, norm=False ..........................................
[CV] .............. alpha=0.13, norm=False, score=0.876, total=   0.4s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:   14.2s remaining:    0.0s
[CV] alpha=0.13, norm=False ..........................................
[CV] .............. alpha=0.13, norm=False, score=0.898, total=   0.4s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:   14.6s remaining:    0.0s
[CV] alpha=0.13999999999999999, norm=True ............................
[CV]  alpha=0.13999999999999999, norm=True, score=0.853, total=   0.4s
[Parallel(n_jobs=1)]: Done  41 out of  41

[CV]  alpha=0.16999999999999998, norm=True, score=0.838, total=   0.3s
[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:   26.8s remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=True ............................
[CV]  alpha=0.16999999999999998, norm=True, score=0.863, total=   0.3s
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:   27.1s remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=False ...........................
[CV]  alpha=0.16999999999999998, norm=False, score=0.886, total=   0.4s
[Parallel(n_jobs=1)]: Done  76 out of  76 | elapsed:   27.5s remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=False ...........................
[CV]  alpha=0.16999999999999998, norm=False, score=0.893, total=   0.4s
[Parallel(n_jobs=1)]: Done  77 out of  77 | elapsed:   27.8s remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=False ...........................
[CV]  alpha=0.16999999999999998, norm=False, score=0.895, total=   0.4s
[Parallel(n_jobs=1)]: Done  78 out of 

[CV]  alpha=0.20999999999999996, norm=False, score=0.892, total=   0.3s
[CV] alpha=0.20999999999999996, norm=False ...........................
[CV]  alpha=0.20999999999999996, norm=False, score=0.896, total=   0.3s
[CV] alpha=0.20999999999999996, norm=False ...........................
[CV]  alpha=0.20999999999999996, norm=False, score=0.876, total=   0.4s
[CV] alpha=0.20999999999999996, norm=False ...........................
[CV]  alpha=0.20999999999999996, norm=False, score=0.898, total=   0.3s
[CV] alpha=0.21999999999999995, norm=True ............................
[CV]  alpha=0.21999999999999995, norm=True, score=0.854, total=   0.3s
[CV] alpha=0.21999999999999995, norm=True ............................
[CV]  alpha=0.21999999999999995, norm=True, score=0.851, total=   0.4s
[CV] alpha=0.21999999999999995, norm=True ............................
[CV]  alpha=0.21999999999999995, norm=True, score=0.871, total=   0.3s
[CV] alpha=0.21999999999999995, norm=True ............................
[C

[CV] . alpha=0.2699999999999999, norm=True, score=0.869, total=   0.3s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.886, total=   0.3s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.891, total=   0.4s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.894, total=   0.3s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.876, total=   0.3s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.899, total=   0.4s
[CV] alpha=0.2799999999999999, norm=True .............................
[CV] . alpha=0.2799999999999999, norm=True, score=0.858, total=   0.3s
[CV] alpha=0.2799999999999999, norm=True .............................
[CV] .

...

[CV]  alpha=0.7899999999999996, norm=False, score=0.893, total=   0.3s
[CV] alpha=0.7899999999999996, norm=False ............................
[CV]  alpha=0.7899999999999996, norm=False, score=0.894, total=   0.3s
[CV] alpha=0.7899999999999996, norm=False ............................
[CV]  alpha=0.7899999999999996, norm=False, score=0.878, total=   0.4s
[CV] alpha=0.7899999999999996, norm=False ............................
[CV]  alpha=0.7899999999999996, norm=False, score=0.901, total=   0.3s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] . alpha=0.7999999999999996, norm=True, score=0.871, total=   0.3s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] . alpha=0.7999999999999996, norm=True, score=0.864, total=   0.4s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] . alpha=0.7999999999999996, norm=True, score=0.882, total=   0.4s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] .

[CV] . alpha=0.8499999999999996, norm=True, score=0.878, total=   0.4s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.889, total=   0.4s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.893, total=   0.4s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.895, total=   0.5s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.878, total=   0.4s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.900, total=   0.4s
[CV] alpha=0.8599999999999995, norm=True .............................
[CV] . alpha=0.8599999999999995, norm=True, score=0.872, total=   0.3s
[CV] alpha=0.8599999999999995, norm=True .............................
[CV] .

[CV] . alpha=0.9099999999999996, norm=True, score=0.881, total=   0.3s
[CV] alpha=0.9099999999999996, norm=True .............................
[CV] . alpha=0.9099999999999996, norm=True, score=0.860, total=   0.3s
[CV] alpha=0.9099999999999996, norm=True .............................
[CV] . alpha=0.9099999999999996, norm=True, score=0.878, total=   0.3s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.889, total=   0.3s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.893, total=   0.3s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.895, total=   0.3s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.878, total=   0.3s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  

[CV] . alpha=0.9699999999999995, norm=True, score=0.874, total=   0.3s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.866, total=   0.4s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.881, total=   0.3s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.861, total=   0.3s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.880, total=   0.3s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  alpha=0.9699999999999995, norm=False, score=0.890, total=   0.3s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  alpha=0.9699999999999995, norm=False, score=0.891, total=   0.3s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  

GridSearchCV(estimator=ComplementNB(),
             param_grid={'alpha': array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ,
       0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31,
       0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42,
       0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53,
       0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64,
       0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75,
       0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86,
       0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97,
       0.98, 0.99]),
                         'norm': [True, False]},
             verbose=100)

In [107]:
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1376  154]
 [ 178 1292]]
              precision    recall  f1-score   support

          -1       0.89      0.90      0.89      1530
           1       0.89      0.88      0.89      1470

    accuracy                           0.89      3000
   macro avg       0.89      0.89      0.89      3000
weighted avg       0.89      0.89      0.89      3000

0.8893333333333333
{'alpha': 0.7499999999999997, 'norm': False}


### SVM

In [83]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(lemmatized_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(lemmatized_tfidf[i])
        y_train.append(labels[i])

In [84]:
svc=SVC()
parameters = {'C': [0.1,1, 10, 100], 'gamma': [1],'kernel': ['rbf','sigmoid']}

In [85]:
clf = GridSearchCV(svc,parameters,cv=3,verbose=10)
clf.fit(x_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.787, total= 3.7min
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.7min remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.781, total= 3.8min
[CV] C=0.1, gamma=1, kernel=rbf ......................................


...

[CV] ............ C=1, gamma=1, kernel=rbf, score=0.908, total= 3.1min
[CV] C=1, gamma=1, kernel=sigmoid ....................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 30.3min remaining:    0.0s


[CV] ........ C=1, gamma=1, kernel=sigmoid, score=0.908, total= 1.8min
[CV] C=1, gamma=1, kernel=sigmoid ....................................
[CV] ........ C=1, gamma=1, kernel=sigmoid, score=0.909, total= 1.6min
[CV] C=1, gamma=1, kernel=sigmoid ....................................
[CV] ........ C=1, gamma=1, kernel=sigmoid, score=0.904, total= 1.4min
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.915, total= 3.1min
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.908, total= 3.3min
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.909, total= 3.3min
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.865, total= 1.1min
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 60.0min finished


GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1],
                         'kernel': ['rbf', 'sigmoid']},
             verbose=10)

In [86]:
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1339  160]
 [  89 1412]]
              precision    recall  f1-score   support

          -1       0.94      0.89      0.91      1499
           1       0.90      0.94      0.92      1501

    accuracy                           0.92      3000
   macro avg       0.92      0.92      0.92      3000
weighted avg       0.92      0.92      0.92      3000

0.917
{'C': 1, 'gamma': 1, 'kernel': 'rbf'}


## Tree_tagged sample

### KNN

In [79]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(tagger_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(tagger_tfidf[i])
        y_train.append(labels[i])

In [80]:
knn=KNeighborsClassifier()
parameters={'n_neighbors':list(range(10,21)),'p':[2],'n_jobs':[-1]}

In [81]:
clf = GridSearchCV(knn,parameters,cv=3,verbose=100)
clf.fit(x_train,y_train)

Fitting 3 folds for each of 11 candidates, totalling 33 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=10, p=2, score=0.705, total=24.5min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 24.5min remaining:    0.0s
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=10, p=2, score=0.732, total=18.0min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 42.6min remaining:    0.0s
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=10, p=2, score=0.703, total=14.6min
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 57.1min remaining:    0.0s
[CV] n_jobs=-1, n_neighbors=11, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=11, p=2, score=0.709, total=14.6min
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapse

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_jobs': [-1],
                         'n_neighbors': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                         20],
                         'p': [2]},
             verbose=100)

In [82]:
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1096  382]
 [ 367 1155]]
              precision    recall  f1-score   support

          -1       0.75      0.74      0.75      1478
           1       0.75      0.76      0.76      1522

    accuracy                           0.75      3000
   macro avg       0.75      0.75      0.75      3000
weighted avg       0.75      0.75      0.75      3000

0.7503333333333333
{'n_jobs': -1, 'n_neighbors': 18, 'p': 2}


### Naive Bayes

In [72]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(tagger_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(tagger_tfidf[i])
        y_train.append(labels[i])

In [75]:
nb=ComplementNB()
parameters={'alpha':np.arange(0.1,1,0.01),'norm':[True,False]}

In [77]:
clf = GridSearchCV(nb,parameters,verbose=100)
clf.fit(x_train,y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.869, total=  15.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.9s remaining:    0.0s
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.855, total=   6.1s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   21.1s remaining:    0.0s
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.875, total=  11.7s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   32.7s remaining:    0.0s
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.846, total=   4.7s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elap

[CV] .............. alpha=0.13, norm=False, score=0.879, total=   4.9s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:  3.3min remaining:    0.0s
[CV] alpha=0.13, norm=False ..........................................
[CV] .............. alpha=0.13, norm=False, score=0.888, total=   5.0s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:  3.4min remaining:    0.0s
[CV] alpha=0.13, norm=False ..........................................
[CV] .............. alpha=0.13, norm=False, score=0.863, total=   4.9s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:  3.4min remaining:    0.0s
[CV] alpha=0.13, norm=False ..........................................
[CV] .............. alpha=0.13, norm=False, score=0.871, total=   5.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  3.5min remaining:    0.0s
[CV] alpha=0.13999999999999999, norm=True ............................
[CV]  alpha=0.13999999999999999, norm=True, score=0.871, total=   4.8s
[Parallel(n_jobs=1)]: Done  41 out of  41

[CV]  alpha=0.16999999999999998, norm=True, score=0.849, total=   6.1s
[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:  6.8min remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=True ............................
[CV]  alpha=0.16999999999999998, norm=True, score=0.860, total=   5.5s
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  6.9min remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=False ...........................
[CV]  alpha=0.16999999999999998, norm=False, score=0.879, total=   5.5s
[Parallel(n_jobs=1)]: Done  76 out of  76 | elapsed:  7.0min remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=False ...........................
[CV]  alpha=0.16999999999999998, norm=False, score=0.879, total=   5.7s
[Parallel(n_jobs=1)]: Done  77 out of  77 | elapsed:  7.1min remaining:    0.0s
[CV] alpha=0.16999999999999998, norm=False ...........................
[CV]  alpha=0.16999999999999998, norm=False, score=0.886, total=   5.5s
[Parallel(n_jobs=1)]: Done  78 out of 

[CV]  alpha=0.20999999999999996, norm=False, score=0.880, total=   5.5s
[CV] alpha=0.20999999999999996, norm=False ...........................
[CV]  alpha=0.20999999999999996, norm=False, score=0.889, total=   5.4s
[CV] alpha=0.20999999999999996, norm=False ...........................
[CV]  alpha=0.20999999999999996, norm=False, score=0.861, total=   5.2s
[CV] alpha=0.20999999999999996, norm=False ...........................
[CV]  alpha=0.20999999999999996, norm=False, score=0.868, total=   5.3s
[CV] alpha=0.21999999999999995, norm=True ............................
[CV]  alpha=0.21999999999999995, norm=True, score=0.869, total=   7.6s
[CV] alpha=0.21999999999999995, norm=True ............................
[CV]  alpha=0.21999999999999995, norm=True, score=0.863, total=   7.9s
[CV] alpha=0.21999999999999995, norm=True ............................
[CV]  alpha=0.21999999999999995, norm=True, score=0.878, total=   8.4s
[CV] alpha=0.21999999999999995, norm=True ............................
[C

[CV] . alpha=0.2699999999999999, norm=True, score=0.864, total=   6.0s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.880, total=   6.0s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.876, total=   6.0s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.889, total=   6.1s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.858, total=   6.0s
[CV] alpha=0.2699999999999999, norm=False ............................
[CV]  alpha=0.2699999999999999, norm=False, score=0.869, total=   6.0s
[CV] alpha=0.2799999999999999, norm=True .............................
[CV] . alpha=0.2799999999999999, norm=True, score=0.867, total=   6.4s
[CV] alpha=0.2799999999999999, norm=True .............................
[CV] .

...

[CV]  alpha=0.7899999999999996, norm=False, score=0.868, total=   6.2s
[CV] alpha=0.7899999999999996, norm=False ............................
[CV]  alpha=0.7899999999999996, norm=False, score=0.879, total=   6.0s
[CV] alpha=0.7899999999999996, norm=False ............................
[CV]  alpha=0.7899999999999996, norm=False, score=0.853, total=   6.0s
[CV] alpha=0.7899999999999996, norm=False ............................
[CV]  alpha=0.7899999999999996, norm=False, score=0.861, total=   6.1s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] . alpha=0.7999999999999996, norm=True, score=0.859, total=   6.0s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] . alpha=0.7999999999999996, norm=True, score=0.855, total=   6.1s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] . alpha=0.7999999999999996, norm=True, score=0.866, total=   6.3s
[CV] alpha=0.7999999999999996, norm=True .............................
[CV] .

[CV] . alpha=0.8499999999999996, norm=True, score=0.854, total=   7.0s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.864, total=   7.1s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.866, total=   6.4s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.876, total=   6.3s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.851, total=   6.2s
[CV] alpha=0.8499999999999996, norm=False ............................
[CV]  alpha=0.8499999999999996, norm=False, score=0.861, total=   6.2s
[CV] alpha=0.8599999999999995, norm=True .............................
[CV] . alpha=0.8599999999999995, norm=True, score=0.855, total=   7.1s
[CV] alpha=0.8599999999999995, norm=True .............................
[CV] .

[CV] . alpha=0.9099999999999996, norm=True, score=0.864, total=   6.7s
[CV] alpha=0.9099999999999996, norm=True .............................
[CV] . alpha=0.9099999999999996, norm=True, score=0.840, total=   6.6s
[CV] alpha=0.9099999999999996, norm=True .............................
[CV] . alpha=0.9099999999999996, norm=True, score=0.854, total=   6.7s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.862, total=   7.0s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.866, total=   7.1s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.875, total=   6.8s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  alpha=0.9099999999999996, norm=False, score=0.849, total=   6.6s
[CV] alpha=0.9099999999999996, norm=False ............................
[CV]  

[CV] . alpha=0.9699999999999995, norm=True, score=0.851, total=   7.0s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.851, total=   6.4s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.864, total=   5.9s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.838, total=   5.8s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.852, total=   6.2s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  alpha=0.9699999999999995, norm=False, score=0.862, total=   6.2s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  alpha=0.9699999999999995, norm=False, score=0.866, total=   6.2s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  

GridSearchCV(estimator=ComplementNB(),
             param_grid={'alpha': array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ,
       0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31,
       0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42,
       0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53,
       0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64,
       0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75,
       0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86,
       0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97,
       0.98, 0.99]),
                         'norm': [True, False]},
             verbose=100)

In [78]:
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1329  142]
 [ 213 1316]]
              precision    recall  f1-score   support

          -1       0.86      0.90      0.88      1471
           1       0.90      0.86      0.88      1529

    accuracy                           0.88      3000
   macro avg       0.88      0.88      0.88      3000
weighted avg       0.88      0.88      0.88      3000

0.8816666666666667
{'alpha': 0.11, 'norm': False}


### SVM

In [100]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(tagger_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(tagger_tfidf[i])
        y_train.append(labels[i])

In [101]:
svc=SVC()
parameters = {'C': [0.1,1, 10, 100], 'gamma': [1],'kernel': ['rbf', 'sigmoid']}

In [102]:
clf = GridSearchCV(svc,parameters,cv=3,verbose=10)
clf.fit(x_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.831, total= 3.8min
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.8min remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.839, total= 3.3min
[CV] C=0.1, gamma=1, kernel=rbf ......................................


...

[CV] ............ C=1, gamma=1, kernel=rbf, score=0.894, total= 2.8min
[CV] C=1, gamma=1, kernel=sigmoid ....................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 27.4min remaining:    0.0s


[CV] ........ C=1, gamma=1, kernel=sigmoid, score=0.896, total= 1.6min
[CV] C=1, gamma=1, kernel=sigmoid ....................................
[CV] ........ C=1, gamma=1, kernel=sigmoid, score=0.899, total= 1.6min
[CV] C=1, gamma=1, kernel=sigmoid ....................................
[CV] ........ C=1, gamma=1, kernel=sigmoid, score=0.891, total= 1.6min
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.905, total= 3.0min
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.907, total= 3.0min
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.894, total= 3.0min
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.861, total= 1.1min
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 56.7min finished


GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1],
                         'kernel': ['rbf', 'sigmoid']},
             verbose=10)

In [103]:
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1330  169]
 [ 105 1396]]
              precision    recall  f1-score   support

          -1       0.93      0.89      0.91      1499
           1       0.89      0.93      0.91      1501

    accuracy                           0.91      3000
   macro avg       0.91      0.91      0.91      3000
weighted avg       0.91      0.91      0.91      3000

0.9086666666666666
{'C': 10, 'gamma': 1, 'kernel': 'rbf'}


## Bi-grams

### KNN

In [22]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(bigrams_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(bigrams_tfidf[i])
        y_train.append(labels[i])

In [23]:
knn=KNeighborsClassifier()
parameters={'n_neighbors':list(range(10,21)),'p':[2],'n_jobs':[-1]}

In [24]:
clf = GridSearchCV(knn,parameters,cv=3,verbose=100)
clf.fit(x_train,y_train)

Fitting 3 folds for each of 11 candidates, totalling 33 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=10, p=2, score=0.740, total=  29.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.0s remaining:    0.0s
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=10, p=2, score=0.746, total=  41.6s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.2min remaining:    0.0s
[CV] n_jobs=-1, n_neighbors=10, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=10, p=2, score=0.758, total=  36.5s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.8min remaining:    0.0s
[CV] n_jobs=-1, n_neighbors=11, p=2 ..................................
[CV] ...... n_jobs=-1, n_neighbors=11, p=2, score=0.745, total=  35.4s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapse

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_jobs': [-1],
                         'n_neighbors': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                         20],
                         'p': [2]},
             verbose=100)

In [25]:
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1124  354]
 [ 318 1204]]
              precision    recall  f1-score   support

          -1       0.78      0.76      0.77      1478
           1       0.77      0.79      0.78      1522

    accuracy                           0.78      3000
   macro avg       0.78      0.78      0.78      3000
weighted avg       0.78      0.78      0.78      3000

0.776
{'n_jobs': -1, 'n_neighbors': 20, 'p': 2}


### Naive Bayes

In [26]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(bigrams_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(bigrams_tfidf[i])
        y_train.append(labels[i])

In [27]:
nb=ComplementNB()
parameters={'alpha':np.arange(0.1,1,0.01),'norm':[True,False]}

In [29]:
clf = GridSearchCV(nb,parameters,cv=3,verbose=100)
clf.fit(x_train,y_train)

Fitting 3 folds for each of 180 candidates, totalling 540 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.853, total=   0.3s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.870, total=   0.2s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[CV] alpha=0.1, norm=True ............................................
[CV] ................ alpha=0.1, norm=True, score=0.860, total=   0.2s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[CV] alpha=0.1, norm=False ...........................................
[CV] ............... alpha=0.1, norm=False, score=0.853, total=   0.3s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elap

[CV]  alpha=0.15999999999999998, norm=True, score=0.852, total=   0.2s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    8.9s remaining:    0.0s
[CV] alpha=0.15999999999999998, norm=True ............................
[CV]  alpha=0.15999999999999998, norm=True, score=0.869, total=   0.2s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:    9.1s remaining:    0.0s
[CV] alpha=0.15999999999999998, norm=True ............................
[CV]  alpha=0.15999999999999998, norm=True, score=0.858, total=   0.2s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:    9.3s remaining:    0.0s
[CV] alpha=0.15999999999999998, norm=False ...........................
[CV]  alpha=0.15999999999999998, norm=False, score=0.851, total=   0.3s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    9.6s remaining:    0.0s
[CV] alpha=0.15999999999999998, norm=False ...........................
[CV]  alpha=0.15999999999999998, norm=False, score=0.868, total=   0.2s
[Parallel(n_jobs=1)]: Done  41 out of  

[CV]  alpha=0.21999999999999995, norm=True, score=0.868, total=   0.2s
[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:   17.5s remaining:    0.0s
[CV] alpha=0.21999999999999995, norm=True ............................
[CV]  alpha=0.21999999999999995, norm=True, score=0.857, total=   0.2s
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:   17.7s remaining:    0.0s
[CV] alpha=0.21999999999999995, norm=False ...........................
[CV]  alpha=0.21999999999999995, norm=False, score=0.851, total=   0.2s
[Parallel(n_jobs=1)]: Done  76 out of  76 | elapsed:   17.9s remaining:    0.0s
[CV] alpha=0.21999999999999995, norm=False ...........................
[CV]  alpha=0.21999999999999995, norm=False, score=0.869, total=   0.2s
[Parallel(n_jobs=1)]: Done  77 out of  77 | elapsed:   18.1s remaining:    0.0s
[CV] alpha=0.21999999999999995, norm=False ...........................
[CV]  alpha=0.21999999999999995, norm=False, score=0.861, total=   0.2s
[Parallel(n_jobs=1)]: Done  78 out of 

[CV] . alpha=0.2899999999999999, norm=True, score=0.860, total=   0.3s
[CV] alpha=0.2899999999999999, norm=False ............................
[CV]  alpha=0.2899999999999999, norm=False, score=0.850, total=   0.3s
[CV] alpha=0.2899999999999999, norm=False ............................
[CV]  alpha=0.2899999999999999, norm=False, score=0.868, total=   0.3s
[CV] alpha=0.2899999999999999, norm=False ............................
[CV]  alpha=0.2899999999999999, norm=False, score=0.861, total=   0.3s
[CV] alpha=0.29999999999999993, norm=True ............................
[CV]  alpha=0.29999999999999993, norm=True, score=0.852, total=   0.3s
[CV] alpha=0.29999999999999993, norm=True ............................
[CV]  alpha=0.29999999999999993, norm=True, score=0.868, total=   0.2s
[CV] alpha=0.29999999999999993, norm=True ............................
[CV]  alpha=0.29999999999999993, norm=True, score=0.860, total=   0.2s
[CV] alpha=0.29999999999999993, norm=False ...........................
[CV]  

[CV] . alpha=0.3899999999999999, norm=True, score=0.850, total=   0.3s
[CV] alpha=0.3899999999999999, norm=True .............................
[CV] . alpha=0.3899999999999999, norm=True, score=0.868, total=   0.3s
[CV] alpha=0.3899999999999999, norm=True .............................
[CV] . alpha=0.3899999999999999, norm=True, score=0.861, total=   0.2s
[CV] alpha=0.3899999999999999, norm=False ............................
[CV]  alpha=0.3899999999999999, norm=False, score=0.850, total=   0.2s
[CV] alpha=0.3899999999999999, norm=False ............................
[CV]  alpha=0.3899999999999999, norm=False, score=0.867, total=   0.3s
[CV] alpha=0.3899999999999999, norm=False ............................
[CV]  alpha=0.3899999999999999, norm=False, score=0.861, total=   0.2s
[CV] alpha=0.3999999999999998, norm=True .............................
[CV] . alpha=0.3999999999999998, norm=True, score=0.849, total=   0.2s
[CV] alpha=0.3999999999999998, norm=True .............................
[CV] .

...

[CV] . alpha=0.6799999999999997, norm=True, score=0.849, total=   0.3s
[CV] alpha=0.6799999999999997, norm=True .............................
[CV] . alpha=0.6799999999999997, norm=True, score=0.865, total=   0.3s
[CV] alpha=0.6799999999999997, norm=True .............................
[CV] . alpha=0.6799999999999997, norm=True, score=0.861, total=   0.3s
[CV] alpha=0.6799999999999997, norm=False ............................
[CV]  alpha=0.6799999999999997, norm=False, score=0.849, total=   0.3s
[CV] alpha=0.6799999999999997, norm=False ............................
[CV]  alpha=0.6799999999999997, norm=False, score=0.866, total=   0.2s
[CV] alpha=0.6799999999999997, norm=False ............................
[CV]  alpha=0.6799999999999997, norm=False, score=0.859, total=   0.2s
[CV] alpha=0.6899999999999997, norm=True .............................
[CV] . alpha=0.6899999999999997, norm=True, score=0.849, total=   0.3s
[CV] alpha=0.6899999999999997, norm=True .............................
[CV] .

[CV]  alpha=0.7699999999999997, norm=False, score=0.865, total=   0.3s
[CV] alpha=0.7699999999999997, norm=False ............................
[CV]  alpha=0.7699999999999997, norm=False, score=0.859, total=   0.2s
[CV] alpha=0.7799999999999997, norm=True .............................
[CV] . alpha=0.7799999999999997, norm=True, score=0.848, total=   0.3s
[CV] alpha=0.7799999999999997, norm=True .............................
[CV] . alpha=0.7799999999999997, norm=True, score=0.863, total=   0.3s
[CV] alpha=0.7799999999999997, norm=True .............................
[CV] . alpha=0.7799999999999997, norm=True, score=0.860, total=   0.3s
[CV] alpha=0.7799999999999997, norm=False ............................
[CV]  alpha=0.7799999999999997, norm=False, score=0.848, total=   0.3s
[CV] alpha=0.7799999999999997, norm=False ............................
[CV]  alpha=0.7799999999999997, norm=False, score=0.865, total=   0.3s
[CV] alpha=0.7799999999999997, norm=False ............................
[CV]  

[CV] . alpha=0.8699999999999996, norm=True, score=0.858, total=   0.2s
[CV] alpha=0.8699999999999996, norm=False ............................
[CV]  alpha=0.8699999999999996, norm=False, score=0.848, total=   0.3s
[CV] alpha=0.8699999999999996, norm=False ............................
[CV]  alpha=0.8699999999999996, norm=False, score=0.865, total=   0.2s
[CV] alpha=0.8699999999999996, norm=False ............................
[CV]  alpha=0.8699999999999996, norm=False, score=0.859, total=   0.3s
[CV] alpha=0.8799999999999996, norm=True .............................
[CV] . alpha=0.8799999999999996, norm=True, score=0.847, total=   0.3s
[CV] alpha=0.8799999999999996, norm=True .............................
[CV] . alpha=0.8799999999999996, norm=True, score=0.864, total=   0.3s
[CV] alpha=0.8799999999999996, norm=True .............................
[CV] . alpha=0.8799999999999996, norm=True, score=0.858, total=   0.3s
[CV] alpha=0.8799999999999996, norm=False ............................
[CV]  

[CV] . alpha=0.9699999999999995, norm=True, score=0.847, total=   0.3s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.863, total=   0.3s
[CV] alpha=0.9699999999999995, norm=True .............................
[CV] . alpha=0.9699999999999995, norm=True, score=0.857, total=   0.3s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  alpha=0.9699999999999995, norm=False, score=0.849, total=   0.3s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  alpha=0.9699999999999995, norm=False, score=0.864, total=   0.3s
[CV] alpha=0.9699999999999995, norm=False ............................
[CV]  alpha=0.9699999999999995, norm=False, score=0.859, total=   0.2s
[CV] alpha=0.9799999999999995, norm=True .............................
[CV] . alpha=0.9799999999999995, norm=True, score=0.847, total=   0.3s
[CV] alpha=0.9799999999999995, norm=True .............................
[CV] .

GridSearchCV(cv=3, estimator=ComplementNB(),
             param_grid={'alpha': array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ,
       0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31,
       0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42,
       0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53,
       0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64,
       0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75,
       0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86,
       0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97,
       0.98, 0.99]),
                         'norm': [True, False]},
             verbose=100)

In [30]:
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1291  226]
 [ 192 1291]]
              precision    recall  f1-score   support

          -1       0.87      0.85      0.86      1517
           1       0.85      0.87      0.86      1483

    accuracy                           0.86      3000
   macro avg       0.86      0.86      0.86      3000
weighted avg       0.86      0.86      0.86      3000

0.8606666666666667
{'alpha': 0.11, 'norm': False}


### SVM

In [31]:
index=list(range(9998))
test_index=random.sample(index,3000)
x_test=[]
y_test=[]
x_train=[]
y_train=[]
for i in range(9998):
    if i in test_index:
        x_test.append(bigrams_tfidf[i])
        y_test.append(labels[i])
    else:
        x_train.append(bigrams_tfidf[i])
        y_train.append(labels[i])

In [32]:
svc=SVC()
parameters = {'C': [0.1,1, 10, 100], 'gamma': [1],'kernel': ['rbf', 'sigmoid']}

In [34]:
clf = GridSearchCV(svc,parameters,cv=3,verbose=100)
clf.fit(x_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.754, total= 4.9min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.9min remaining:    0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.757, total= 4.8min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  9.8min remaining:    0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.766, total= 4.9min
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 14.7min remaining:    0.0s
[CV] C=0.1, gamma=1, kernel=sigmoid ..................................
[CV] ...... C=0.1, gamma=1, kernel=sigmoid, score=0.840, total= 4.7min
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1],
                         'kernel': ['rbf', 'sigmoid']},
             verbose=100)

In [35]:
predictions=clf.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
print(clf.best_params_)

[[1288  220]
 [ 177 1315]]
              precision    recall  f1-score   support

          -1       0.88      0.85      0.87      1508
           1       0.86      0.88      0.87      1492

    accuracy                           0.87      3000
   macro avg       0.87      0.87      0.87      3000
weighted avg       0.87      0.87      0.87      3000

0.8676666666666667
{'C': 10, 'gamma': 1, 'kernel': 'rbf'}
